# Introdução 

O objetivo deste notebook é a estatistica. Seja ela descritiva ou regressiva. Fazendo algumas definições, citando alguns teoremas importantes, ajustar dados, propor hipóteses e testa-las. 

A base de dados utilizada será a base PeNSE 2015 do [IBGE](https://www.ibge.gov.br/) sobre saude e educação, disponível em: 

> https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html?caminho=pense/2015/microdados/

A amostra que vamos utilizar é a `PeNSE_2015_AMOSTRA2.zip`

## PeNSE (Pesquisa Nacional de Saúde do Escolar)

A pesquisa PeNSE é uma pesquisa feita por amostragem que permite conhecer e dimensionar os fatores de risco e proteção à saúde dos adolescente , é feita utilizando dados para a seleção de escolas publicas e privadas do Instituto Nacional de Estudos e Pesquisas Educacionais Anísio Teixeira - INEP.

É fruto de uma parceria entre os Ministérios da Saúde e Educação. A pesquisa fornece informações sobre características básicas da população de estudo, incluindo aspectos socioecnômicos, como escolaridade dos pais, inserção no mercado de trabalho e posse de bens e serviços; fatores de risco comportamentais relacionados a hábitos alimentares, sedentarismo, tabagismo, consumo de álcool e outras drogas; saúde sexual e reprodutiva; exposição a acidentes e violências; hábitos de higiene; saúde bucal; saúde mental;... dentre vaŕias outras informações. 

---

## Notas Metodológicas

---

## Importações

In [ ]:
import pandas as pd 
import numpy as np 